In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras import layers, ops
from sklearn.model_selection import train_test_split

2025-11-19 16:58:34.959542: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 16:58:34.960079: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-19 16:58:35.037041: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-19 16:58:36.844602: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
## Config a fournir aux etudiants.
maxlen = 128
embed_dim = 256
num_heads = 8
ff_dim = 1024

1. Chargez votre dataset, et previsualisez un echantillon de celui-ci.

In [3]:
csv_path = "dataset.csv"

df = pd.read_csv(csv_path)

df = df.dropna(subset=["seq", "sst3"]).reset_index(drop=True)
df = df[df["seq"].str.len() == df["sst3"].str.len()].reset_index(drop=True)

sequences = df["seq"].astype(str).tolist()
labels_ss = df["sst3"].astype(str).tolist()

print(df.head(5))


  pdb_id chain_code                              seq  \
0   1MZW          B  EVKASLRALGEPITLFGEGPAERRERLRNIL   
1   1P9I          A  MDQLNALLASLEAENKQLKAKVEELLAKVGE   
2   3HFE          B  GARGSNTIGARLNRVEDKVTQLDQRLALITD   
3   3LJM          C  *EWEALEKKCAALESKLQALEKKLEALEHG*   
4   4PND          B  *GNILQKIENILKKIENILWKIENILQKIEG   

                              sst3  len  
0  CHHHHHHHCCCCCEECCCCHHHHHHHHHHHC   31  
1  CCCHHHHHHHHHHHHHHHHHHHHHHHHHHCC   31  
2  CCCCCCCHHHHHHHHHHHHHHHHHHHHHHCC   31  
3  CCHHHHHHHHHHHHHHHHHHHHHHHHHHCCC   31  
4  CCCHHHHHHHHHHHHHHHHHHHHHHHHHHHC   31  


2. Encodez les acides amines ainsi que les structures secondaires correspondantes.

In [4]:
aa_chars = sorted({c for seq in sequences for c in seq})

aa_to_idx = {c: i + 1 for i, c in enumerate(aa_chars)}
idx_to_aa = {i + 1: c for i, c in enumerate(aa_chars)}
vocab_size = len(aa_to_idx) + 1

ss_to_idx = {"H": 0, "E": 1, "C": 2}
idx_to_ss = {v: k for k, v in ss_to_idx.items()}
num_classes = 3

encoded_seqs = [[aa_to_idx.get(c, 0) for c in seq] for seq in sequences]

encoded_ss = [[ss_to_idx.get(c, 2) for c in ss] for ss in labels_ss]

3. Preparez vos vecteurs de train et prediction (X et y).

In [5]:
X = keras.utils.pad_sequences(
    encoded_seqs,
    maxlen=maxlen,
    padding="post",
    truncating="post",
    value=0,
)

y = keras.utils.pad_sequences(
    encoded_ss,
    maxlen=maxlen,
    padding="post",
    truncating="post",
    value=-1,
)

x_train, x_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

4. Creez une classe TransformerBlock comprenant deux methodes, init() et call().

In [6]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

6. Creez une classe TokenAndPositionEmbedding comprenant trois methodes, init(), call(), compute_mask

In [7]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(
            input_dim=vocab_size,
            output_dim=embed_dim,
            mask_zero=True,
        )
        self.pos_emb = layers.Embedding(
            input_dim=maxlen,
            output_dim=embed_dim,
        )

    def call(self, x):
        seq_len = ops.shape(x)[-1]
        positions = ops.arange(start=0, stop=seq_len, step=1)
        pos = self.pos_emb(positions)
        tok = self.token_emb(x)
        return tok + pos

    def compute_mask(self, inputs, mask=None):
        return self.token_emb.compute_mask(inputs)

7. Implementez votre modele avec un layer d'embedding puis deux block Transformers consecutifs et un layer dense de sortie s'appuyant sur la fonction d'activation softmax. Vous pourrez afficher votre model avec model.summary() 

In [8]:
inputs = keras.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)

x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="transformer")
model.summary()

2025-11-19 16:58:38.281105: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2025-11-19 16:58:38.281156: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-11-19 16:58:38.281162: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: madeleine
2025-11-19 16:58:38.281165: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] hostname: madeleine
2025-11-19 16:58:38.281321: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] libcuda reported version is: 580.95.5
2025-11-19 16:58:38.281342: I external/local_xla/xla/stream_executor/cuda/

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, 128, 256)       │        38,400 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 128, 256)       │     2,630,144 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ (None, 128, 256)       │     2,630,144 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128, 3)         │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,299,459 (20.22 MB)

 Trainable params: 5,299,459 (20.22 MB)

 Non-trainable params: 0 (0.00 B)

8. Redigez deux methodes loss et accuracy afin d'ignorer le padding des sequences passees dans le modele.

In [9]:
def loss(y_true, y_pred):
    mask = ops.cast(y_true >= 0, "float32")
    y_true_clipped = ops.maximum(y_true, 0)

    scce = keras.losses.SparseCategoricalCrossentropy(
        from_logits=False,
        reduction="none",
    )
    loss_per_token = scce(y_true_clipped, y_pred)
    loss_per_token = loss_per_token * mask

    return ops.sum(loss_per_token) / (ops.sum(mask) + 1e-9)


def accuracy(y_true, y_pred):
    mask = ops.cast(y_true >= 0, "float32")
    y_true_clipped = ops.maximum(y_true, 0)

    y_pred_labels = ops.argmax(y_pred, axis=-1)
    matches = ops.cast(ops.equal(y_true_clipped, y_pred_labels), "float32")
    matches = matches * mask
    return ops.sum(matches) / (ops.sum(mask) + 1e-9)

9. Preparez des callbacks (earlyStopping, ajustement du learning rate), puis compilez votre modele et entrainez le.

In [10]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
    )
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=loss,
    metrics=[accuracy],
)

history = model.fit(
    x_train,
    y_train,
    batch_size=16,
    epochs=30,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
)

Epoch 1/30
269/269 ━━━━━━━━━━━━━━━━━━━━ 72s 248ms/step - accuracy: 0.5151 - loss: 1.0053 - val_accuracy: 0.5206 - val_loss: 0.9718 - learning_rate: 3.0000e-04
Epoch 2/30
269/269 ━━━━━━━━━━━━━━━━━━━━ 66s 246ms/step - accuracy: 0.5372 - loss: 0.9500 - val_accuracy: 0.5377 - val_loss: 0.9534 - learning_rate: 3.0000e-04
Epoch 3/30
269/269 ━━━━━━━━━━━━━━━━━━━━ 66s 246ms/step - accuracy: 0.5431 - loss: 0.9410 - val_accuracy: 0.5355 - val_loss: 0.9488 - learning_rate: 3.0000e-04
Epoch 4/30
269/269 ━━━━━━━━━━━━━━━━━━━━ 67s 248ms/step - accuracy: 0.5453 - loss: 0.9383 - val_accuracy: 0.5437 - val_loss: 0.9358 - learning_rate: 3.0000e-04
Epoch 5/30
269/269 ━━━━━━━━━━━━━━━━━━━━ 67s 248ms/step - accuracy: 0.5484 - loss: 0.9336 - val_accuracy: 0.5478 - val_loss: 0.9328 - learning_rate: 3.0000e-04
Epoch 6/30
269/269 ━━━━━━━━━━━━━━━━━━━━ 67s 248ms/step - accuracy: 0.5522 - loss: 0.9275 - val_accuracy: 0.5430 - val_loss: 0.9361 - learning_rate: 3.0000e-04
Epoch 7/30
269/269 ━━━━━━━━━━━━━━━━━━━━ 67s 24

10. Enregistrez les courbes d'apprentissage dans training_panels.png

In [11]:
def plot_history_panels(history, out_path="training_panels.png"):

    train_loss = history.history.get("loss", [])
    val_loss = history.history.get("val_loss", [])
    train_acc = history.history.get("accuracy", [])
    val_acc = history.history.get("val_accuracy", [])

    epochs = range(1, len(train_loss) + 1)

    fig, axs = plt.subplots(2, 1, figsize=(8, 8))

    ax = axs[0]
    ax.plot(epochs, train_loss, label="Train loss")
    ax.plot(epochs, val_loss, label="Val loss", linestyle="--")
    ax.set_title("Loss")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.grid(True)
    ax.legend()

    ax = axs[1]
    ax.plot(epochs, train_acc, label="Train accuracy")
    ax.plot(epochs, val_acc, label="Val accuracy", linestyle="--")
    ax.set_title("Accuracy")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Accuracy")
    ax.grid(True)
    ax.legend()

    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()
    print(f"Panel sauvegardé sous : {out_path}")

plot_history_panels(history, out_path="training_panels.png")

Panel sauvegardé sous : training_panels.png


11. Ecrire une fonction permettant de tirer un echantillon au hasard de notre dataset, et d'effectuer une prediction a partir de la sequence d'AA. Comparez votre prediction avec la ss3 reelle.

In [12]:
def predict_random_sequence(model, df, maxlen, aa_to_idx, idx_to_ss):
    row = df.sample(1).iloc[0]
    seq = row["seq"]
    true_ss = row["sst3"]

    print("\n=== Exemple tiré au hasard ===")
    print("Séquence :", seq)
    print("Structure réelle :", true_ss)

    encoded = [aa_to_idx.get(c, 0) for c in seq]
    encoded = keras.utils.pad_sequences(
        [encoded], maxlen=maxlen, padding="post", truncating="post"
    )

    pred = model.predict(encoded)[0]
    pred_labels = pred.argmax(axis=-1)
    pred_ss3 = "".join(idx_to_ss[i] for i in pred_labels)

    print("Structure prédite :", pred_ss3[:len(seq)])
    print("=============================\n")

    return seq, true_ss, pred_ss3[:len(seq)]


predict_random_sequence(model, df, maxlen, aa_to_idx, idx_to_ss)


=== Exemple tiré au hasard ===
Séquence : GIQPVISAQEQETQIVLYGKLVEARQKHANKMDVPPAILATNKILVDMAKMRPTTVENVKRIDGVSEGKAAMLAPLLEVIKHFCQTNSVQTDLFSSTKPQEEQ
Structure réelle : CCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCHHHHCCHHHHHHHHHHCCCCHHHHCCCCCCCHHHHHHCHHHHHHHHHHHHHCCCCCCCCCCCCCCCCC
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
Structure prédite : CCCCCCCHHHHHHHHHHHHHHHHHHHHHHCHCCCCCCEEHHCHHHHHHHHCCCCCHHEEEECCCCCHHHHHHHHHHHHHHHHHHCCCCCCCCEEECCCCCCCC



('GIQPVISAQEQETQIVLYGKLVEARQKHANKMDVPPAILATNKILVDMAKMRPTTVENVKRIDGVSEGKAAMLAPLLEVIKHFCQTNSVQTDLFSSTKPQEEQ',
 'CCCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHCCCHHHHCCHHHHHHHHHHCCCCHHHHCCCCCCCHHHHHHCHHHHHHHHHHHHHCCCCCCCCCCCCCCCCC',
 'CCCCCCCHHHHHHHHHHHHHHHHHHHHHHCHCCCCCCEEHHCHHHHHHHHCCCCCHHEEEECCCCCHHHHHHHHHHHHHHHHHHCCCCCCCCEEECCCCCCCC')